In [ ]:
# Define the complexity function
def get_complexity(obj):
    llama_num = obj['code_llama_success']
    gpt3_num = obj['method2_gpt3_5_success']
    gpt4_num = obj['method2_gpt4_success']
    sum = llama_num + gpt3_num + gpt4_num
    if (llama_num + gpt3_num >= 7) or llama_num == 5:
        return 1
    elif gpt3_num == 5:
        return 2
    elif gpt4_num == 5:
        return 3
    elif gpt4_num >= 2 or gpt3_num == 2:
        return 4
    else:
      return 5

Combine data from Code Llama and GPT 3.5 and GPT 4

In [ ]:
import json

# Paths to the input files
file2_path = "./mbpp_label_refined_gpt3_4_238.jsonl"
file1_path = "./mbpp_label_refined_llama238.jsonl"
file3_path = "./mbpp_test_5class_completion.jsonl"

# Open both files for reading and writing
with open(file1_path, 'r') as file1, open(file2_path, 'r') as file2:
    # Read lines from both files
    lines_file1 = file1.readlines()
    lines_file2 = file2.readlines()

    # Iterate through each line in both files
    for index, line_file1 in enumerate(lines_file1):
        # Extract the 'code_llama_success' property from file1
        data = json.loads(line_file1)
        code_llama_success = data.get('code_llama_success')

        # Update the corresponding line in file2 with 'code_llama_success' from file1
        data_file2 = json.loads(lines_file2[index])
        data_file2['code_llama_success'] = code_llama_success
        lines_file2[index] = json.dumps(data_file2) + '\n'

# Write the updated content back to file2
with open(file3_path, 'w') as file2:
    file2.writelines(lines_file2)

Remove the noise - all data points with GPT-4, GPT-3.5, and Code Llama succeeding 0 times

In [ ]:
import json

# Path to the input JSONL file
input_file_path = "./mbpp_label_refined_all.jsonl"
output_file_path = "./mbpp_label_refined_no_zero.jsonl"

# Read the input file and filter rows
with open(input_file_path, 'r') as file:
    # Read lines from the file
    lines = file.readlines()

    # Filter rows where all three keys have a value of 0
    filtered_lines = [line for line in lines if json.loads(line).get('code_llama_success', 0) != 0
                      or json.loads(line).get('method2_gpt3_5_success', 0) != 0
                      or json.loads(line).get('method2_gpt4_success', 0) != 0]

# Write the filtered content back to the file
with open(output_file_path, 'w') as file:
    file.writelines(filtered_lines)

In [ ]:
import json
# Path to the input JSONL file
input_file_path = "./mbpp_label_no_zero.jsonl"
file3_path = "./mbpp_label_refined_no_zero_complexity.jsonl"

# Read the input file and process each line
with open(input_file_path, 'r') as file:
    # Read lines from the file
    lines = file.readlines()

    # Process each line
    for index, line in enumerate(lines):
        # Load JSON object from the line
        data = json.loads(line)

        # Append 'complexity' key with the associated value
        data['complexity'] = get_complexity(data)

        # Update the line with the appended 'complexity' key
        lines[index] = json.dumps(data) + '\n'

# Write the updated content back to the file
with open(file3_path, 'w') as file:
    file.writelines(lines)

Format the data for OpenAI fine-tuning

In [ ]:
import json

input_file_path = './mbpp_label_refined_no_zero_complexity.jsonl'  # Path to the provided JSONL file
output_file_path = './mbpp_test_5class_completion.jsonl'  # Path for the output JSONL file

# Read the JSONL file
with open(input_file_path, 'r') as file:
    current_data = [json.loads(line) for line in file]

# Perform the transformation
transformed_data = []

for item in current_data:
    transformed_entry = {
        "prompt": f"On a scale of 1 to 5 based solely on the complexity of creating the correct code for the task, "
                  f"where 1 represents a very easy task, 2 represents a fairly straightforward task, and 3 represents "
                  f"a medium difficulty task, 4 represents a challenging task, and 5 represents "
                  f"a highly complex problem, the complexity of the task \"{item['text']}\" is: ",
        "completion": str(item["complexity"])
    }
    transformed_data.append(transformed_entry)

# Write the transformed data to a new JSONL file
with open(output_file_path, 'w') as outfile:
    for entry in transformed_data:
        json_record = json.dumps(entry)
        outfile.write(f"{json_record}\n")

Finally, we output the test-train-validation files for the fine-tuning. To be used in the fine_tuning.py file

In [ ]:
# Define the paths for output files
output_file_1 = "./5class_train.jsonl"
output_file_2 = "./5class_val.jsonl"
output_file_3 = "./5class_test.jsonl"

# Path to the input JSONL file
input_file_path = "./mbpp_test_5class_completion.jsonl"

# Read the input file and divide into sections
with open(input_file_path, 'r') as file:
    # Read lines from the file
    lines = file.readlines()

    # test-train-validation split #1
    # section_1 = lines[:120]
    # section_2 = lines[120:140]
    # section_3 = lines[140:174]

    # test-train-validation split #2
    # section_1 = lines[170:] + lines[10:40] + lines[60:90] + lines[100:150] + lines[154:160]
    # section_2 = lines[90:100] + lines[160:170]
    # section_3 = lines[40:60] + lines[0:10] + lines[150:154]

    # test-train-validation split #3
    # section_1 = lines[:80] + lines[134:]
    # section_2 = lines[80:100]
    # section_3 = lines[100:134]

    # test-train-validation split #4
    # section_1 = lines[0:116] + lines[150:154]
    # section_2 = lines[154:174]
    # section_3 = lines[116:150]

    # # line breakup 5
    section_1 = lines[170:] + lines[10:40] + lines[50:60] + lines[70:100] + lines[110:150] + lines[154:160]
    section_2 = lines[100:110] + lines[160:170]
    section_3 = lines[40:50] + lines[60:70] + lines[0:10] + lines[150:154]


# Write each section to different output files
with open(output_file_1, 'w') as file:
    file.writelines(section_1)

with open(output_file_2, 'w') as file:
    file.writelines(section_2)

with open(output_file_3, 'w') as file:
    file.writelines(section_3)


Below are functions that were not used in the generation / testing of data but were used in the analysis of the distribution of complexity in the dataset.

In [ ]:
# Code to show distribution of complexity graphically in dataset
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

# Sample data (replace this with your array of integers)
data = global_arr  # Replace this with your array of integers

# Create a histogram
plt.hist(data, bins=10, alpha=0.7, color='blue')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of Integer Data')
plt.grid(True)
plt.show()

# Calculate summary statistics
mean = np.mean(data)
median = np.median(data)
std_dev = np.std(data)
min_val = np.min(data)
max_val = np.max(data)
mode_val = stats.mode(data)

twenty = np.percentile(data, 20)
forty = np.percentile(data, 40)
seventy = np.percentile(data, 70)

def get_quartiles(arr):
  return twenty, forty, seventy

print(f"Mean: {mean}")
print(f"Median: {median}")
print(f"Mode: {mode_val}")
print(f"Standard Deviation: {std_dev}")
print(f"Minimum Value: {min_val}")
print(f"Maximum Value: {max_val}")
print(f" twenty: {twenty}")
print(f"forty {forty}")
print(f"seventy {seventy}")



In [ ]:
# Function to Count Occurrences of Complexity in Dataset
import json

def count_complexity_occurrences(file_path):
    complexity_counts = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            complexity = data.get('completion')
            # Check if 'complexity' exists and is in the range 1-5
            if complexity is not None and int(complexity) in range(1, 6):
                complexity_counts[int(complexity)] += 1

    return complexity_counts

# Replace 'file_path.jsonl' with the path to your JSONL file
result = count_complexity_occurrences('mbpp_test_5class_completion.jsonl')

print("Complexity Counts:")
for complexity, count in result.items():
    print(f"Complexity {complexity}: {count} occurrences")
